In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
file=pd.read_csv("../Database/ready_for_sql/joined_data.csv")
df=file.copy()
df.head()

,channel_id,custom_url,topic_category,channel_view_count,subscriber_count,channel_video_count,video_id,published_at,video_length,like_count,comment_count,view_count
0,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,lmH5uqwaFq8,2022-11-01 07:00:15,2:59,19653,0,3425275
1,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,0SY0Yn0yF9o,2022-10-29 07:00:00,29:52:00,15076,0,2882582
2,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,sNyF7BvVfxs,2022-10-25 07:00:16,2:49,47763,0,8673081
3,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,K4kqqCzF-BA,2022-10-22 07:00:12,1:01:21,57936,0,11744611
4,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,gfZmvllWVwY,2022-10-18 07:00:19,2:39,46041,0,8775011


In [3]:
#make new column that has binary classification. if view count is greather than 1,000,000 then add 1 if less than add 0
def viral(row):
    if row['view_count'] > 1000000:
        return 1
    else:
        return 0

In [4]:
#add the viral column 
df['viral']=df.apply(lambda row: viral(row), axis=1)
df.tail()

,channel_id,custom_url,topic_category,channel_view_count,subscriber_count,channel_video_count,video_id,published_at,video_length,like_count,comment_count,view_count,viral
8599,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,xRi8SGcRDOY,2022-03-28 15:00:24,9,4,0,311,0
8600,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,C-34pIsWZPk,2022-03-27 15:00:04,4:49,166,79,4642,0
8601,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,L2GdB1gB1ZM,2022-03-26 15:00:09,16,133,7,5091,0
8602,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,89IewFGQQ6E,2022-03-25 18:00:03,16,1,0,31,0
8603,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,QS32cM-BQ6M,2022-03-24 15:00:11,11,2,0,76,0


In [5]:
#turn published at to day of week published
df["datetime"]=pd.to_datetime(df['published_at'])
df['day_of_week_published']=df['datetime'].dt.day_name()
df

,channel_id,custom_url,topic_category,channel_view_count,subscriber_count,channel_video_count,video_id,published_at,video_length,like_count,comment_count,view_count,viral,datetime,day_of_week_published
0,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,lmH5uqwaFq8,2022-11-01 07:00:15,2:59,19653,0,3425275,1,2022-11-01 07:00:15,Tuesday
1,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,0SY0Yn0yF9o,2022-10-29 07:00:00,29:52:00,15076,0,2882582,1,2022-10-29 07:00:00,Saturday
2,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,sNyF7BvVfxs,2022-10-25 07:00:16,2:49,47763,0,8673081,1,2022-10-25 07:00:16,Tuesday
3,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,K4kqqCzF-BA,2022-10-22 07:00:12,1:01:21,57936,0,11744611,1,2022-10-22 07:00:12,Saturday
4,UCbCmjCuTUZos6Inko4u57UQ,@cocomelon,Music,142468175305,146000000,811,gfZmvllWVwY,2022-10-18 07:00:19,2:39,46041,0,8775011,1,2022-10-18 07:00:19,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8599,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,xRi8SGcRDOY,2022-03-28 15:00:24,9,4,0,311,0,2022-03-28 15:00:24,Monday
8600,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,C-34pIsWZPk,2022-03-27 15:00:04,4:49,166,79,4642,0,2022-03-27 15:00:04,Sunday
8601,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,L2GdB1gB1ZM,2022-03-26 15:00:09,16,133,7,5091,0,2022-03-26 15:00:09,Saturday
8602,UCC7jlYxfWti7WAW8r7ef1RQ,@anyonecantravel,Hobby,184778,829,109,89IewFGQQ6E,2022-03-25 18:00:03,16,1,0,31,0,2022-03-25 18:00:03,Friday


In [6]:
#remove columns not needed for ml
df=df.drop(['channel_id','custom_url','topic_category','video_id','published_at','datetime'],axis=1)
df

,channel_view_count,subscriber_count,channel_video_count,video_length,like_count,comment_count,view_count,viral,day_of_week_published
0,142468175305,146000000,811,2:59,19653,0,3425275,1,Tuesday
1,142468175305,146000000,811,29:52:00,15076,0,2882582,1,Saturday
2,142468175305,146000000,811,2:49,47763,0,8673081,1,Tuesday
3,142468175305,146000000,811,1:01:21,57936,0,11744611,1,Saturday
4,142468175305,146000000,811,2:39,46041,0,8775011,1,Tuesday
...,...,...,...,...,...,...,...,...,...
8599,184778,829,109,9,4,0,311,0,Monday
8600,184778,829,109,4:49,166,79,4642,0,Sunday
8601,184778,829,109,16,133,7,5091,0,Saturday
8602,184778,829,109,16,1,0,31,0,Friday


In [7]:
#drop video length- causing problems in ml
df=df.drop(['video_length'],axis=1)
df

,channel_view_count,subscriber_count,channel_video_count,like_count,comment_count,view_count,viral,day_of_week_published
0,142468175305,146000000,811,19653,0,3425275,1,Tuesday
1,142468175305,146000000,811,15076,0,2882582,1,Saturday
2,142468175305,146000000,811,47763,0,8673081,1,Tuesday
3,142468175305,146000000,811,57936,0,11744611,1,Saturday
4,142468175305,146000000,811,46041,0,8775011,1,Tuesday
...,...,...,...,...,...,...,...,...
8599,184778,829,109,4,0,311,0,Monday
8600,184778,829,109,166,79,4642,0,Sunday
8601,184778,829,109,133,7,5091,0,Saturday
8602,184778,829,109,1,0,31,0,Friday


In [8]:
#drop day of week puplished- going to keep df with it and creating new df without it (for other ml models)
df1=df.drop(['day_of_week_published'],axis=1)
df1

,channel_view_count,subscriber_count,channel_video_count,like_count,comment_count,view_count,viral
0,142468175305,146000000,811,19653,0,3425275,1
1,142468175305,146000000,811,15076,0,2882582,1
2,142468175305,146000000,811,47763,0,8673081,1
3,142468175305,146000000,811,57936,0,11744611,1
4,142468175305,146000000,811,46041,0,8775011,1
...,...,...,...,...,...,...,...
8599,184778,829,109,4,0,311,0
8600,184778,829,109,166,79,4642,0
8601,184778,829,109,133,7,5091,0
8602,184778,829,109,1,0,31,0


Logistic Regression

In [9]:
#seperate features and target
y=df1['viral']
X=df1.drop(columns='viral')

In [10]:
#split data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(6453, 6)

In [11]:
#create logistic regression model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

#fit train the model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [12]:
#make predictions
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,1,1
1,0,0
2,1,0
3,0,0
4,1,0
5,1,0
6,1,0
7,1,1
8,1,0
9,1,0


In [13]:
#accuracy score
print(accuracy_score(y_test, y_pred))

0.5416085541608554


In [14]:
#classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.44      0.60      1694
           1       0.31      0.94      0.46       457

    accuracy                           0.54      2151
   macro avg       0.64      0.69      0.53      2151
weighted avg       0.82      0.54      0.57      2151



Random Forest Classifier


In [15]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [16]:
#pip install 
#%pip install imbalanced-learn==0.9.0
#%pip install scikit-learn==1.0

In [17]:
# Create our features
X = pd.get_dummies(df.drop(columns="viral"))

# Create our target
y = df["viral"]

In [18]:
X.describe()

,channel_view_count,subscriber_count,channel_video_count,like_count,comment_count,view_count,day_of_week_published_Friday,day_of_week_published_Monday,day_of_week_published_Saturday,day_of_week_published_Sunday,day_of_week_published_Thursday,day_of_week_published_Tuesday,day_of_week_published_Wednesday
count,8.604000e+03,8.604000e+03,8604.000000,8.604000e+03,8604.00000,8.604000e+03,8604.000000,8604.000000,8604.000000,8604.000000,8604.000000,8604.000000,8604.000000
mean,6.967666e+09,1.618893e+07,6393.563226,8.606447e+04,3962.32973,3.786546e+06,0.159228,0.121688,0.109019,0.101697,0.173059,0.150511,0.184798
std,1.738527e+10,2.969994e+07,27295.399522,4.459795e+05,21020.67703,2.988759e+07,0.365910,0.326944,0.311681,0.302267,0.378320,0.357593,0.388156
min,0.000000e+00,1.600000e+01,0.000000,0.000000e+00,0.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.273273e+06,7.230000e+03,165.000000,2.900000e+01,3.00000,1.075000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.132332e+08,2.640000e+06,427.000000,1.361500e+03,62.00000,4.933850e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.315880e+09,1.450000e+07,1406.000000,2.005925e+04,835.00000,6.370585e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.424682e+11,1.460000e+08,246686.000000,1.451987e+07,728578.00000,2.073001e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
# Check the balance of our target values
y.value_counts()

0    6778
1    1826
Name: viral, dtype: int64

In [20]:
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=1)

X_train.shape

(6453, 13)

In [21]:
# Create a random forest classifier
from imblearn.ensemble import BalancedRandomForestClassifier

model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fitting the model
model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [22]:
# Calculated the balanced accuracy score
y_pred=model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

0.9997074312463429

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00      1709
          1       1.00      1.00      1.00      1.00      1.00      1.00       442

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      2151



In [24]:
# List the features sorted in descending order by feature importance
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.4375234106899802, 'view_count'),
 (0.25460518302414575, 'like_count'),
 (0.13467827401253188, 'comment_count'),
 (0.08341003198375474, 'subscriber_count'),
 (0.0622909284973609, 'channel_view_count'),
 (0.02245115627376809, 'channel_video_count'),
 (0.0013346292368818606, 'day_of_week_published_Friday'),
 (0.0009761054185375899, 'day_of_week_published_Wednesday'),
 (0.0008005311288398117, 'day_of_week_published_Thursday'),
 (0.0007975241976709164, 'day_of_week_published_Saturday'),
 (0.00047264001044753344, 'day_of_week_published_Tuesday'),
 (0.000395412108282185, 'day_of_week_published_Monday'),
 (0.0002641734177985688, 'day_of_week_published_Sunday')]

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1708,    1],
       [   0,  442]])

Easy Ensemble Classifier

In [26]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model_eec=EasyEnsembleClassifier(n_estimators=100, random_state=1)
model_eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [27]:
# Calculated the balanced accuracy score
y_pred=model_eec.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

1.0

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[1709,    0],
       [   0,  442]])

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00      1709
          1       1.00      1.00      1.00      1.00      1.00      1.00       442

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      2151

